In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [ ]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

vllamadaEmpresa.shape


In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

In [ ]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

In [ ]:
def siPrimerCero(y):
    if len(y) <= 4:
        return "NONE"
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

In [ ]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [1206]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Aramark"
Pais = "Chile"
baseFinal = True
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,Rut,Dv,rut2,rut1,nombres,Apellido Paterno,Apellido Materno,apellidos,nacimiento,email,Tipo de Carga,empleador,poliza,rutE
0,26201429,0,26201429-0,26201429-0,Gabriel Jose,Patiño,Patiño,Patiño Patiño,1989-03-31,gabrielpat89@gmail.com,tittular,45329,76.178.390-4,76.178.390-4
1,25902630,k,25902630-k,25902630-k,Ruben Javier,Bonillo,Arismendi,Bonillo Arismendi,1989-07-06,rubenbonillo89@gmail.com,titular,45329,76.178.390-4,76.178.390-4
2,26538234,7,26538234-7,26538234-7,Jose Alejandro,Lopez,Brito,Lopez Brito,1978-04-02,josealopeb78@gmail.com,titular,45329,76.178.390-4,76.178.390-4
3,25289018,1,25289018-1,25289018-1,Elizabeth,Caicedo,NaN,Caicedo,1983-01-04,caicedoelizabeth52@gmail.com,titular,45329,76.178.390-4,76.178.390-4
4,27218389,9,27218389-9,27218389-9,Clara Marlene,Lifonso,Ilizarbe,Lifonso Ilizarbe,1977-04-05,Claramarlenelifonso@gmail.com,titular,45329,76.178.390-4,76.178.390-4


In [1207]:
Pacientes_Archivo.shape

(10, 14)

In [1208]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        #"Id Contacto",
    ]
]

In [1209]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("-", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("–", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(" ", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\1490231206.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\1490231206.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\1490231206.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["ru

(10, 15)

In [1210]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(10, 16)

In [1211]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┐", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("═", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("╔", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┌", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┐", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("═", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("╔", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┌", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\3410721566.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\3410721566.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [1212]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
#Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [1213]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")


In [ ]:
Pacientes_Archivo["Llave"] = Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]
Pacientes_Archivo["PolizaLlave"] = Pacientes_Archivo["poliza"] +  Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]

#Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/eliminar2.xlsx", index=False)

PAcientes Programas


In [ ]:
""" query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head() """

account contact relation


In [ ]:
""" query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head() """

empresa contacto convenio


In [1214]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c, Empresa__c, Paciente__c,N_Documento_Titular__c, Relaci_n_Con_Asegurado__c, Identificador__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__c,Paciente__c,N_Documento_Titular__c,Relaci_n_Con_Asegurado__c,Identificador__c
0,a40RN0000010edJYAQ,163820064,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002kCcTAAA0,163820064,Titular,Aramark Servicios Mineros Y Remotos Ltda.
1,a40RN0000010gujYAA,255356798,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002amVmqAAE,255356798,Titular,Aramark Servicios Mineros Y Remotos Ltda.
2,a40RN0000010lxzYAA,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.
3,a40RN0000010lyHYAQ,130949185,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002anxOWAAY,130949185,Titular,Aramark Servicios Mineros Y Remotos Ltda.
4,a40RN0000013XgPYAU,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.


In [1215]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
        "Empresa__r.P_liza__c":"Poliza",
        "Empresa__r.Rut_Empresa__c":"Rut Empresa",
        "Relaci_n_Con_Asegurado__c":"Relación",
        "Identificador__c":"Nombre Empresa",
        "Campa_a__c":"Campaña",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Relación,Nombre Empresa
0,a40RN0000010edJYAQ,163820064,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002kCcTAAA0,163820064,Titular,Aramark Servicios Mineros Y Remotos Ltda.
1,a40RN0000010gujYAA,255356798,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002amVmqAAE,255356798,Titular,Aramark Servicios Mineros Y Remotos Ltda.
2,a40RN0000010lxzYAA,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.
3,a40RN0000010lyHYAQ,130949185,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002anxOWAAY,130949185,Titular,Aramark Servicios Mineros Y Remotos Ltda.
4,a40RN0000013XgPYAU,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.


In [1216]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()

empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("-", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("–", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(" ", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.upper()
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.strip()

empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("-", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("–", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(".", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(" ", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.upper()
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.strip()


empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\3330564967.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\3330564967.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\3330564967.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when r

(219, 9)

In [1217]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Relación,Nombre Empresa,Rut_y
0,a40RN0000010edJYAQ,163820064,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002kCcTAAA0,163820064,Titular,Aramark Servicios Mineros Y Remotos Ltda.,163820064
1,a40RN0000010gujYAA,255356798,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002amVmqAAE,255356798,Titular,Aramark Servicios Mineros Y Remotos Ltda.,255356798
2,a40RN0000010lxzYAA,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.,202201466
3,a40RN0000010lyHYAQ,130949185,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002anxOWAAY,130949185,Titular,Aramark Servicios Mineros Y Remotos Ltda.,130949185
4,a40RN0000013XgPYAU,202201466,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000005OgWJYA0,202201466,Titular,Aramark Servicios Mineros Y Remotos Ltda.,202201466


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

Rescatamos todos los ruts Id con su empresa desde la campaña


In [1218]:
#Pacientes_Archivo.head()

Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [1219]:
print(campaña)

Aramark


In [1220]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [1221]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(10, 27)

In [1222]:
#Pacientes_con_id.head()
#Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [1223]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    

In [1224]:
noestan = pd.DataFrame()
if ("nacimiento" in noestan.columns) == True:  
    Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
    Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    Pacientes_con_id.head()
noestan = Pacientes_con_id[(Pacientes_con_id["RutEstaoNo"] == False)]

In [1225]:
#noestan.to_excel("F:/Empresas SF/reportes/noestan.xlsx", index=False)
noestan.head()

,Rut,Dv,rut2,rut1,nombres,Apellido Paterno,Apellido Materno,apellidos,nacimiento,email,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto


In [1226]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
#Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]

# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Subir_pacientes["rut1"] = noestan["rut"]
Subir_pacientes["rut2"] = noestan["rutT"]

if ("nombres" in noestan.columns) == True:
    Subir_pacientes["FirstName"] = noestan["nombres"]
if ("apellidos" in noestan.columns) == True:
    Subir_pacientes["LastName"] = noestan["apellidos"]
if ("rut" in noestan.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        noestan["rut"].astype(str).str.upper()
    )


if ("email" in noestan.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = noestan["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in noestan.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    Subir_pacientes["derty"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in noestan.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    noestan.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = noestan["sexo"]
if ("direccion" in noestan.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        noestan["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in noestan.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        noestan["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in noestan.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        noestan["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = noestan["cargo"].str.title()

if ("division" in noestan.columns) == True:
    Subir_pacientes["PersonDepartment"] = noestan["division"].str.title()

if ("celular" in noestan.columns) == True:
    Subir_pacientes["Phone"] = (
        noestan["celular"].astype(str).str.strip().replace("+", "")
        
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

#Subir_pacientes_Que_no_Estan = Subir_pacientes
#Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
#Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]

#Subir_pacientes["PersonBirthdate"] = Subir_pacientes["PersonBirthdate"].str.replace("0001-01-01 00:00:00", "")

# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)
""" Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
) """
Subir_pacientes.head()

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [1227]:
Subir_pacientes.shape

(0, 14)

In [1228]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc where IdentificationId__pc = '" + rut + "')"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
   

In [1229]:
Subir_pacientes.head()

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [1230]:
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)


Subir_pacientes.shape

(0, 14)

In [1231]:
""" Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape """

' Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)\nSubir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]\n# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()\nSubir_pacientes.shape '

ACA SE CREAN LOS PACIENTES Q NO ESTAN

In [1232]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15596\1983134162.py:57: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))


In [ ]:
""" Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
) """
#Subir_pacientes["Id_creado"]

In [1234]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "' order by CreatedDate"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]

df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()
df2.shape

100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


(5, 12)

In [1235]:
#RELACION EMPRESA PACIENMTES ---- EMPRESACONTACTOCONVENIOS


sinId = pd.DataFrame()
Subir_pacientes["Id_creado"] = Subir_pacientes["Id_creado"].astype(str)
sinId = Subir_pacientes[Subir_pacientes["Id_creado"].astype(str).str.len() < 12]
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id_creado"].astype(str).str.len() >= 12]
sinId.shape

(0, 16)

In [1236]:
sinId.head()

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,dato_account,Id_creado


In [1237]:
sinId["Id_creado"] = sinId["rut1"].apply(lambda y: traerId(y))

Subir_pacientes = pd.concat([Subir_pacientes, sinId], axis=0)
Subir_pacientes.shape

(0, 16)

In [1238]:
""" Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/dertyuiop.xlsx", index=False
)
Subir_pacientes.head() """

' Subir_pacientes.to_excel(\n    "F:/Empresas SF/reportes/dertyuiop.xlsx", index=False\n)\nSubir_pacientes.head() '

In [1239]:
Subir_pacientes = Subir_pacientes[
    [
        "rut1",
        "rut2",
        "Id_creado",
        
    ]
]





In [1240]:
Subir_pacientes.shape

(0, 3)

In [1241]:
Pacientes_con_id["Id_creado"] = Pacientes_con_id["Id Account Paciente"]

In [1242]:
Pacientes_con_id.shape

(10, 28)

In [1243]:

#noestan["Id Account Paciente"] = np.where(Subir_pacientes["IdentificationId__pc"] == noestan["rut"],Subir_pacientes["Id_creado"] + "SI","NO")
todos_con_Id = pd.merge(
    left=Pacientes_con_id,
    right=Subir_pacientes,
    how="left",
    left_on="rut",
    right_on="rut1",
)

todos_con_Id = todos_con_Id[
    [
        "poliza",
        #"empleador",
        "rutE",
        "rut",
        "rutT",
        "Id_creado_x",
        "Id_creado_y",
    ]
]

todos_con_Id.shape


(10, 6)

In [1244]:
todos_con_Id.head()

,poliza,rutE,rut,rutT,Id_creado_x,Id_creado_y
0,761783904,761783904,262014290,262014290,001RN000006DUkxYAG,NaN
1,761783904,761783904,25902630K,25902630K,001RN000006DYBlYAO,NaN
2,761783904,761783904,265382347,265382347,0018c00002ay50nAAA,NaN
3,761783904,761783904,252890181,252890181,001RN000006PRkoYAG,NaN
4,761783904,761783904,272183899,272183899,001RN000006PLx7YAG,NaN


In [1245]:
todos_con_Id.Id_creado_x.fillna(todos_con_Id["Id_creado_y"], inplace=True)
todos_con_Id.Id_creado_x.fillna("FARSO", inplace=True)
todos_con_Id = todos_con_Id[todos_con_Id["Id_creado_x"] != "FARSO"]
todos_con_Id2 = todos_con_Id[todos_con_Id["Id_creado_x"] == "FARSO"]
todos_con_Id2["Id_creado_x"] = todos_con_Id2["rut"].apply(lambda y: traerId(y))
todos_con_Id = pd.concat([todos_con_Id, todos_con_Id2], axis=0)

In [1246]:
todos_con_Id.head()

,poliza,rutE,rut,rutT,Id_creado_x,Id_creado_y
0,761783904,761783904,262014290,262014290,001RN000006DUkxYAG,NaN
1,761783904,761783904,25902630K,25902630K,001RN000006DYBlYAO,NaN
2,761783904,761783904,265382347,265382347,0018c00002ay50nAAA,NaN
3,761783904,761783904,252890181,252890181,001RN000006PRkoYAG,NaN
4,761783904,761783904,272183899,272183899,001RN000006PLx7YAG,NaN


In [1247]:
""" todos_con_Id.to_excel(
    "F:/Empresas SF/reportes/todos_con_Id.xlsx", index=False
) """


' todos_con_Id.to_excel(\n    "F:/Empresas SF/reportes/todos_con_Id.xlsx", index=False\n) '

In [1248]:
if campaña == "Southbridge Plan Mujer":
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":

    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Cruz Verde":
    
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    
    VempresaConvenio = "a3yRN0000006sORYAY"
elif campaña == "Talo":
    
    VempresaConvenio = "a3yRN0000006t10YAA"
else:
    
    VempresaConvenio = "Buscarv"


In [1249]:
if(campaña == "MetLife" or campaña == "BCI Seguros Colectivos"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["poliza"] + todos_con_Id["rutT"] + todos_con_Id["rut"]
    
elif(campaña == "Vida Security"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] +empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["poliza"] + todos_con_Id["rutE"] + todos_con_Id["rutT"] + todos_con_Id["rut"]
else:
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["rutT"] + todos_con_Id["rut"]    
    

todos_con_Id["esta"] = todos_con_Id["llave"].isin(empresacontactoconvenio_PacientesSF["llave"])
empresacontactoconvenio_PacientesSF["Noesta"] = empresacontactoconvenio_PacientesSF["llave"].isin(todos_con_Id["llave"])


diccionario = df2[
    [
        "P_liza__c",
        "Id",        
    ]
]
diccionario = diccionario.drop_duplicates(
    [
        "P_liza__c",
        "Id",        
    ],
    keep="last",
)
diccionario = diccionario.drop_duplicates(
    [
        "P_liza__c",        
    ],
    keep="last",
)

diccionario = diccionario.set_index("P_liza__c").T.to_dict(orient="list")

diccionario2 = df2[
    [
        "P_liza__c",
        "Activo__c",        
    ]
]
diccionario2 = diccionario2.drop_duplicates(
    [
        "P_liza__c",        
    ],
    keep="last",
)

diccionario2 = diccionario2.set_index("P_liza__c").T.to_dict(orient="list")


todos_con_Id["N_Documento_Titular__c"] = todos_con_Id["rutT"]
todos_con_Id["Paciente__c"] = todos_con_Id["Id_creado_x"]
todos_con_Id["Empresa__c"] = VempresaConvenio
todos_con_Id["EmpActiva"] = "revisar"


todos_con_Id = todos_con_Id[
    [
        "poliza",
        "rutE",
        "rut",
        "rutT",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esta",
        "EmpActiva",
    ]
]


if (VempresaConvenio == "Buscarv"):
    todos_con_Id["poliza2"] = todos_con_Id["poliza"] 
    todos_con_Id.replace({"poliza2": diccionario}, inplace=True)    
    todos_con_Id["Empresa__c"] = np.where(todos_con_Id["poliza2"].str.len() >= 15, todos_con_Id["poliza2"], todos_con_Id["Empresa__c"])
    todos_con_Id["EmpActiva"] = todos_con_Id["poliza"] 
    todos_con_Id.replace({"EmpActiva": diccionario2}, inplace=True)
    
todos_con_Id = todos_con_Id[
    [
        "poliza",        
        "rutE",
        "rut",
        "rutT",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esta",
        "EmpActiva",
    ]
]


In [1250]:
todos_con_Id.head(50)

,poliza,rutE,rut,rutT,Paciente__c,N_Documento_Titular__c,Empresa__c,esta,EmpActiva
0,761783904,761783904,262014290,262014290,001RN000006DUkxYAG,262014290,a3yRN0000006rUmYAI,True,True
1,761783904,761783904,25902630K,25902630K,001RN000006DYBlYAO,25902630K,a3yRN0000006rUmYAI,True,True
2,761783904,761783904,265382347,265382347,0018c00002ay50nAAA,265382347,a3yRN0000006rUmYAI,True,True
3,761783904,761783904,252890181,252890181,001RN000006PRkoYAG,252890181,a3yRN0000006rUmYAI,True,True
4,761783904,761783904,272183899,272183899,001RN000006PLx7YAG,272183899,a3yRN0000006rUmYAI,True,True
5,761783904,761783904,279399382,279399382,001RN000006PKOLYA4,279399382,a3yRN0000006rUmYAI,True,True
6,761783904,761783904,250667515,250667515,001RN000006PIb2YAG,250667515,a3yRN0000006rUmYAI,True,True
7,761783904,761783904,256180502,256180502,0018c00002b0AM5AAM,256180502,a3yRN0000006rUmYAI,True,True
8,761783904,761783904,260279173,260279173,001RN000006PUkFYAW,260279173,a3yRN0000006rUmYAI,True,True
9,761783602,761783602,179287870,179287870,0018c00002bHWzzAAG,179287870,a3yRN0000006rUlYAI,False,True


In [1251]:
todos_con_Id.to_excel(
    "F:/Empresas SF/reportes/todos con.xlsx", index=False
)

In [1252]:

empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/empresacontacto.xlsx", index=False
)

Cuadrar eliminar pacientes q no estan en el listado
//-----------------------------------------------------------//


In [1253]:
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "163820064")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "153298939")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "191710940")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "182470082")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "HEGJ930928HMNRRL06")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16007258K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "86676540")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "202201466")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "105233469")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "130949185")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "255356798")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "34005945")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "185410072")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "197415029")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "254468134")].index, inplace=True)



empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Relación,Nombre Empresa,Rut_y,llave,Noesta
8,a40RN0000014uPdYAI,262014290,76117696K,76117696K,a3yRN0000006rOBYAY,001RN000006DUkxYAG,262014290,Titular,Aramark Servicios Mineros Y Remotos Ltda.,262014290,262014290262014290,True
9,a40RN0000018WgTYAU,27591679K,76117696K,76117696K,a3yRN0000006rOBYAY,0018c00002l990VAAQ,27591679K,Titular,Aramark Servicios Mineros Y Remotos Ltda.,27591679K,27591679K27591679K,False
10,a40RN0000014rLJYAY,122573230,761783602,761783602,a3yRN0000006rUlYAI,001RN0000057xyEYAQ,122573230,Titular,Central De Restaurantes Aramark Ltda.,122573230,122573230122573230,False
11,a40RN0000014rLKYAY,199137158,761783602,761783602,a3yRN0000006rUlYAI,001RN000006ED5EYAW,122573230,Carga,Central De Restaurantes Aramark Ltda.,199137158,122573230199137158,False
12,a40RN0000014rLLYAY,186731832,761783602,761783602,a3yRN0000006rUlYAI,001RN000006EB8EYAW,122573230,Carga,Central De Restaurantes Aramark Ltda.,186731832,122573230186731832,False


In [1254]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [1255]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

'0038c00003HhmVJAAZ'

In [1256]:
# contactd["PersonContactId"]
print(datetime.now())

2024-02-27 18:03:39.447148


In [1257]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/tod  es.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [1258]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

' if campaña != "MetLife":\n    pacientes_en_campaña = pd.merge(\n        left=campaña_seleccionada,\n        right=Pacientes_con_id,\n        how="left",\n        left_on="Id Account",\n        right_on="Id Account Paciente",\n    )\n    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"\n    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"\n    pacientes_en_campaña.to_excel(\n        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False\n    )\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False\n    )\n '

In [1259]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

' if campaña != "MetLife1":\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    if ("poliza" in campaña_en_pacientes.columns) == True:\n        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)\n\n    campaña_en_pacientes = campaña_en_pacientes.rename(\n        columns={"Programas_por_Empresas__c": "programas por empresa"}\n    )\n    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]\n    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[\n        "programas por empresa"\n    ]\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False\n    ) '

In [1260]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [1261]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

NameError: name 'campaña_seleccionada' is not defined

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()